In [21]:
import os
import torch
import torch.optim as optim
import torch.utils.data as Dataset

from Load import load_shot, load_eig, load_dist
from FAUSTDataset import FAUSTDataset
from ResNet import DFM

In [22]:
data_dir = '../data/'
train_reg_shot = 'training/reg_shot/'
train_scan_shot = 'training/scan_shot/'
train_eigen = 'training/reg_lb/'
train_dist = 'training/l2_dist/'
test_scan_shot = 'test/scan_shot/'
test_eigen = 'test/scan_lb/'
inter = 'inter_challenge.txt'
intra = 'intra_challenge.txt'

In [23]:
train=True
batch_size=32
lr = 0.001
max_epoch=50
eignum = 120

In [24]:
# load training data
shot = []
eigen = []
dist = []
if train:
    streig = "%d" % (eignum)
    for i in range(100):
        strnum = '%03d' % (i)
#                 data_dir, train_reg_shot, 
        fn = ''.join([data_dir, train_reg_shot, 'tr_reg_res_', strnum, '.txt'])
        shot.append(load_shot(fn))
        fn_eig = ''.join([data_dir, train_eigen, 'tr_reg_', streig, '_', strnum, '.h5'])
        eigen.append(load_eig(fn_eig))
#         fn_dist = ''.join([data_dir, train_dist, 'tr_reg_dist_', strnum, '.h5'])
#         dist.append(load_dist(fn_dist))
print('loaded all')
    
train_dataset = FAUSTDataset(train, shot, eigen, dist)

train_loader = Dataset.DataLoader(dataset=train_dataset, batch_size=batch_size,
                          shuffle=True, num_workers=1, drop_last=False
                         )

# import model
model = DFM(train)

# optimizer
optimizer = optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-08)

# loss
criterion = EuclideanLoss()

for epoch in range(0, max_epoch):
    if epoch % 100 > 0:
        epoch *= 0.1
        
    total_loss = 0
    model.train()
    for data in enumerate(train_loader):
        src_shot, tar_shot, src_eigen, tar_eigen = data
        optimizer.zero_grad()
        # P: soft corr matrix
        P, C = model(src_shot, tar_shot, src_eigen, tar_eigen)
        loss = criterion(P, src_dist, tar_dist)
        total_loss += loss
        loss.backward()
        optimizer.step()
        
    print("[{0}/{1}] Loss: {2}".format(epoch, max_epoch, total_loss/batch_size))
    
    if epoch % 10 == 0:
        torch.save(model.state_dict(), os.path.join('{}.pth'.format(epoch)))

loaded all


TypeError: __init__() missing 4 required positional arguments: 'src_shot', 'tar_shot', 'src_eigen', and 'tar_eigen'